# ba2a - Implement MotifEnumeration

In [1]:
from itertools import product

def hamming_distance(s,t):
    cnt = 0
    for i,j in zip(s,t):
        cnt += i!=j
    return cnt

def motif(dna,kmer,k,d):
    inTotalCnt = 0
    for seq in dna:
        inSeqCnt = 0
        for i in range(len(seq)-k+1):
            if hamming_distance(seq[i:i+k],kmer)<=d:
                inSeqCnt += 1
        if inSeqCnt>=1:
            inTotalCnt +=1
    if inTotalCnt==len(dna):
        return True
    return False

def ba2a(dna,k,d):
    # find all the kmers
    all_kmers = [''.join(i) for i in list(product('ACGT',repeat=k))]
    dna = dna.split('\n')
    ans = []
    # now find the list where a kmers from all_kmers list is in all the seq
    for kmer in all_kmers:
        if motif(dna,kmer,k,d)==True:
            ans.append(kmer)
    return list(set(ans))

# input

k = 5
d = 1

dna = '''TCTTGAGAAGGCCGGAATCAAAACC
GATGCTTGCAAGAAGAATCAGGTAG
AATCAGACGTTGCCAGTGGGAAGTG
TGTAGTCATGCGAAGCGGTGAACCA
ACATCTCCCGTTTGAAACCATGGGC
AGTCGGACGACACGCAAGCACACTG
ACGAGCACTGAAACAAAGCTTACGT
CTTGCCAGCTCGTTGAAGCACGTGG
TCCTGGGGTCAAAATAAACAATTGT
AAGCACCTCATAAGTGCTTCGAAGC'''

# output

for i in ba2a(dna,k,d):
    print(i)

AATCA
AACCA
AAACA
AAGCA


# ba2b - Find a Median String

In [11]:
# sum of minimum mismatches but in every string is the score
# find the minimum score

In [15]:
def minimum_score(dna,kmer,k):
    score = 0
    for seq in dna:
        min_hd = float('inf')
        for i in range(len(seq)-k+1):
            cur_hd = hamming_distance(seq[i:i+k],kmer)
            if cur_hd<min_hd:
                min_hd = cur_hd
        score += min_hd
    return score

def ba2b(dna,k):
    dna = dna.split('\n')
    min_map = {}
    for seq in dna:
        for i in range(len(seq)-k+1):
            min_map[seq[i:i+k]] = minimum_score(dna,seq[i:i+k],k)
    # return min_map
    ans = ''
    min_score = float('inf')
    for k,v in zip(min_map.keys(),min_map.values()):
        if v<min_score:
            min_score = v
            ans = k
    return ans


    
k = 3
dna = '''AAATTGACGCAT
GACGACCACGTT
CGTCAGCGCCTG
GCTGAGCACCGG
AGTACGGGACAG'''

print(ba2b(dna,k))

GAC


# ba2c - Find a Profile-most Probable k-mer in a String

In [64]:
def probability(kmer,pm):
    pro = 1
    for i in range(len(kmer)):
        pro *= pm[kmer[i]][i]
    return pro

def ba2c(dna,k,pm):
    best_pro = -1*float('inf')
    ans = ''
    for i in range(len(dna)-k+1):
        pro = probability(dna[i:i+k],pm)
        if pro>best_pro:
            ans = dna[i:i+k]
            pro = best_pro
    return ans

dna = 'ACCTGTTTATTGCCTAAGTTCCGAACAAACCCAATATAGCCCGAGGGCCT'
k = 5
pm = {
'A': [0.2,0.2,0.3,0.2,0.3],
'C': [0.4,0.3,0.1,0.5,0.1],
'T': [0.3,0.3,0.5,0.2,0.4],
'G': [0.1,0.2,0.1,0.1,0.2]
}

print(ba2c(dna,k,pm))

GGCCT


# ba2d - Implement GreedyMotifSearch

In [59]:
# max-probab, min-score

def probab(pm,kmer):
    pro = 1
    for i in range(len(kmer)):
        pro *= pm[kmer[i]][i]
    return pro

def profile_matrix(res,k):
    pm = {
        'A':[0]*k,
        'C':[0]*k,
        'G':[0]*k,
        'T':[0]*k
    }
    for kmer in res:
        for i in range(len(kmer)):
            pm[kmer[i]][i] += 1/len(res)
    return pm

def score(kmers,k):
    score = 0
    for i in range(k):
        l = [j[i] for j in kmers]
        score += (len(l)-max(l.count('A'),l.count('C'),l.count('G'),l.count('T')))
    return score


def most_probable(kmer,rest,k):
    res = []
    res.append(kmer)
    for seq in rest:
        pm = profile_matrix(res,k)
        max_prob = -1*float('inf')
        ans = ''
        for i in range(len(seq)-k+1):
            prob = probab(pm,seq[i:i+k])
            if prob>max_prob:
                max_prob = prob
                ans = seq[i:i+k]
        res.append(ans)
    return res

def ba2d(k,n,dna):
    dna = dna.split('\n')
    template = dna[0]
    rest = dna[1:n]
    # find all the template kmers (all the kmers in the first line of the dna :3 ..)
    all_temp = []
    for i in range(len(template)-k+1):
        all_temp.append(template[i:i+k])
    # return all_temp
    kmers = []
    for kmer in all_temp:
        target = most_probable(kmer,rest,k)
        # print(target)
        # target will be a array of kmer+n-1 other kmers
        kmers.append(target)
    # print(all_temp)
    min_score = float('inf')
    ans = ''
    # print(kmers)
    for target in kmers:
        sc = score(target,k)
        if sc < min_score:
            min_score = sc
            ans = target
    return ans


n = 5
k = 3
dna = '''GGCGTTCAGGCA
AAGAATCAGTCA
CAAGGAGTTCGC
CACGTCAATCAC
CAATAATATTCG'''

for i in ba2d(k,n,dna):
    print(i)

CAG
CAG
CAA
CAA
CAA


In [31]:
# l = ['abc','def']
# k = 3
# for i in range(k):
#     print([j[i] for j in l])

# ba2e - Implement GreedyMotifSearch with Pseudocounts

In [61]:
# max-probab, min-score
# change with d is only the pm matrix intialization in d it was 0 but here it will 1/4

def probab(pm,kmer):
    pro = 1
    for i in range(len(kmer)):
        pro *= pm[kmer[i]][i]
    return pro

def profile_matrix(res,k):
    pm = {
        'A':[1/4]*k,
        'C':[1/4]*k,
        'G':[1/4]*k,
        'T':[1/4]*k
    }
    for kmer in res:
        for i in range(len(kmer)):
            pm[kmer[i]][i] += 1/len(res)
    return pm

def score(kmers,k):
    score = 0
    for i in range(k):
        l = [j[i] for j in kmers]
        score += (len(l)-max(l.count('A'),l.count('C'),l.count('G'),l.count('T')))
    return score


def most_probable(kmer,rest,k):
    res = []
    res.append(kmer)
    for seq in rest:
        pm = profile_matrix(res,k)
        max_prob = -1*float('inf')
        ans = ''
        for i in range(len(seq)-k+1):
            prob = probab(pm,seq[i:i+k])
            if prob>max_prob:
                max_prob = prob
                ans = seq[i:i+k]
        res.append(ans)
    return res

def ba2d(k,n,dna):
    dna = dna.split('\n')
    template = dna[0]
    rest = dna[1:n]
    # find all the template kmers (all the kmers in the first line of the dna :3 ..)
    all_temp = []
    for i in range(len(template)-k+1):
        all_temp.append(template[i:i+k])
    # return all_temp
    kmers = []
    for kmer in all_temp:
        target = most_probable(kmer,rest,k)
        # print(target)
        # target will be a array of kmer+n-1 other kmers
        kmers.append(target)
    # print(all_temp)
    min_score = float('inf')
    ans = ''
    # print(kmers)
    for target in kmers:
        sc = score(target,k)
        if sc < min_score:
            min_score = sc
            ans = target
    return ans


n = 5
k = 3
dna = '''GGCGTTCAGGCA
AAGAATCAGTCA
CAAGGAGTTCGC
CACGTCAATCAC
CAATAATATTCG'''

for i in ba2d(k,n,dna):
    print(i)

TTC
ATC
TTC
ATC
TTC


# ba2f - Implement RandomizedMotifSearch

In [55]:
# first select a kmers (array of kmer from every line randomly)
# the calculate pm and probability for select the best from every line
# then return that as new kmers
# continue do it till getting the best score (minimum score)
# if break before 1000 iter, do it again (store it)
# use 1/4 type pseudo count

In [53]:
import random

def profile_matrix(kmers,k):
    pm = {
        'A':[1/4]*k,
        'C':[1/4]*k,
        'G':[1/4]*k,
        'T':[1/4]*k
    }
    for kmer in kmers:
        for i in range(len(kmer)):
            pm[kmer[i]][i] += 1/len(kmers)
    return pm

def probability(kmer,pm):
    pro = 1
    for i in range(len(kmer)):
          pro *= pm[kmer[i]][i]
    return pro

def update_kmers(dna,pm,k):
    up_kmers = []
    for seq in dna:
        max_pro = -1*float('inf')
        ans = ''
        for i in range(len(seq)-k+1):
            pro = probability(seq[i:i+k],pm)
            if pro>max_pro:
                max_pro=pro
                ans = seq[i:i+k]
        up_kmers.append(ans)
    return up_kmers

def score(kmers,k):
    score = 0
    for i in range(k):
        l = [j[i] for j in kmers]
        score += (len(l)-max(l.count('A'),l.count('C'),l.count('G'),l.count('T')))
    return score


def ba2e(dna,k,n):
    dna = dna.split('\n')
    ans_tl = []
    for itter in range(1001):
        kmers = []
        for seq in dna:
                i = random.choice(range(0,k))
                kmers.append(seq[i:i+k])
        best_score = float('inf')
        ans = ''
        while True:
            pm = profile_matrix(kmers,k)
            kmers = update_kmers(dna,pm,k)
            sc = score(kmers,k)
            if sc<best_score:
                best_score=sc
                ans = kmers
            else:
                break
        ans_tl.append((ans,best_score))
    # return ans_tl
    mini = float('inf')
    ans = ''
    for i in ans_tl:
        if i[1]<mini:
            mini=i[1]
            ans = i[0]
    return ans

    
    
            

n = 5
k = 8

dna = '''CGCCCCTCTCGGGGGTGTTCAGTAAACGGCCA
GGGCGAGGTATGTGTAAGTGCCAAGGTGCCAG
TAGTACCGAGACCGAAAGAAGTATACAGGCGT
TAGATCAAGTTTCAGGTGCACGTCGGTGAACC
AATCCACCAGCTCCACGTGCAATGTTGGCCTA'''

for i in ba2e(dna,k,n):
    print(i)

TCTCGGGG
CCAAGGTG
TACAGGCG
TTCAGGTG
TCCACGTG


# ba2g - Implement GibbsSampler

In [65]:
# NO NEED ...

# ba2h - Implement DistanceBetweenPatternAndStrings

In [63]:
# sum of min hamming distance in all the seq

def hamming_distance(s,t):
    count = 0
    for i,j in zip(s,t):
        count += i!=j
    return count

def ba2h(kmer,dna):
    dna = dna.split(' ')
    l = []
    for seq in dna:
        min_hd = float('inf')
        for i in range(len(seq)-len(kmer)+1):
            hd = hamming_distance(seq[i:i+len(kmer)],kmer)
            if hd<min_hd:
                min_hd = hd
        l.append(min_hd)
    return sum(l)

kmer = 'ACTGCGT'
dna = 'GACCATGCACCTTTTGGCGTCGTCCAAATTGCCAGAGGTTTTAAATCACCGGACCGCCCTTTGAGGACGGCTTTGTGATTATCACTTTGGACGCCCAACGAATATGAGA GTTGCGTGCATAGTTGTATAATTAAGTGGTATAGCACGAAAATCACGAGACCTCGGTGTTGCATCTACCCATTGCCTGGGGCGAACCTGTGTATCTCTGGCCACACACA CGAGCGAGAGACTCCCCTGTCAACTCGCGACAGTAAGATACATAAAGCCACCACCTCCATGTGGCTAAAGTCAGCCATTTATACGGGTAGATCATATGCAGACAGCGTG GAGGTTGGGTCCATTGAATTCCCGGACTTTCCTCGTTATACGAATGATAACGCGAATGCAAATTTGGTCGGCTAAGTAGTAGCTAGACGAAAAATTGTTAGCCCGTCGA AGCTCCACCAGCATGATTAGACACAAGCCCGGAAACAGGACCGAATAACGTGAATACAAGCGGTCGTCTAAGAAGGTTGTGACAAGTATTTTGTAGATCGGGAGGTACA CGACAGGCCTTCTAAATTGTTTAAAGGCCGCACGTGGGTAAATTACGCCCTCTAGCTATTGAGCAACCCAGAATGCCGGGTAGATGACGATCTCCTCACGTATCGCACA TGAACAATGCAGAGCCATTTCTGGATTTTATCCTCTCTCGCATATAACGGGTGGACACCTAATGGCCACGAAATCGTATGATAGTTTTCTGAAGTGTCAGCTATTTGCG TGCCGCGAGAGAGGCCACCCACGTCTCGCCCCGGGATCGACGATGATTATTCCGGTACCAAGGTTGTTTCAAATTTCTGACTCCCCCTATACTATGAGCCAACATCATT CGCAGGGACCAGCCGGACATAATGAGCTCCACGGCGCGCTTTAGTTGCGGGCCGGGGGTGCTATCCATGGTTGTTCACGGTATGCGAGCAGAGGACTGAACGCATTAGG AACTGCAAGAGAGAGTTCCTCCCAAGGATCCGTGTTGAGTTAACCAGGTCTCTCTTTGAATCCCTGGCCGGCGCCGTTAATCGACCGAGCCCACGTGTGGCTGACGCAC ACCTTTTGCACTGTTCGTGTATCATAGAGTACCTGGCCTCGAGAGCGTTGCAGCGGGACAGAGTGATTCTACATGCGGACCTGGACACCGATGTAGTCTGCCTTAATCA CCCAGGAAAGCCGAGCTGGACCAGCCCGCAACCCGGATGTCCAACACGTTGTAGCTGGATTGGAGCCGCTGAAGACCAATATTTTCTCCCCAGCCCGGCAAGGTTGAAC ACCCCTTGAACTACGTTAATGGGCGGCCAAGAACTGCTTAAGAAAAGCATACTCGCAATTCAGCGTTGCGGCCCGCAGAGATGTTAAGAGGAATTTCTAAGAGGGACTG GCTCGAGAGCTACCGCTTAAAAGCAGACCCAAGTCTACAATTCTATCTATATTCCACCATGATCCGGGTCGACGGATTGGGCGAGCGGTCAGGGAACGGTTGTATAGGT GACCCGTATGCTCGATCTTACAACGTATTAATGCGTGGGAGAGCGAAATCTGAGATGTGCGGTTAGGTGGCTTAGACGCAGATCCAGTGCAAGCGGATACGAACGGCAA GCGCTGGGACGAGCCACCGGTTATAACGCAGAGACCGGAACCCACGGATATTACACTGCCGAGGAGCTGTAACAGCCGAAAGCCCGAACGGCTAAAACTCGGATTGTGG TCGTAGCCCTCTTGCGGGGTGTTCGGCACGAAGAGCGATCTGAGTCAGGACGCGATCACCGCACGACTTAATCCCGCCTTTTAAAATGCCTTACTTTCACACTAGGCTT GTAGTAGCCAGTTGGGTAAAATGAGATGTGCGATCTGTCTCCACGAGTGACGATTTAGAGCCACAACCACCAAATGGACCCCCGGGGGTATTTCGCAACATGGCTACCC GAGACGAATTAATCTGCGACGCCCCAGAGTCTCTACCGGAAGCGATGCTCAACTTTTTGCCTCCGGTGTAGTTATCGCATTACCCCGGAAAATCGTTCACGACGGCGTA TTAATCAAAGCTGGGGGCAGCGATGTCCGGGGATTATGGCACTCCGGTCATGCTGCCCGGGCTGCTCGTTTGTGTACTCCTGTCGATAGGTCTACTTTATACCCAACTG GCACTCGTCTAGCCTAAGTTAGTTCATGCCAACCTCACACGTGGGTCCCCGAAGGATGCAATGACTGACTTAACATCAAAAGCTAGGCACTCGGACATCCCGTTTTGGT TACACACATTTTCAGGCATTCATCTGGTCAATGTAACAGAAACGCCGTTCTGTTAAAAGCGCCCTGATTCGTAACGTACAACTACCATTGACCACAAACCGTGACAATA GGTGTGGGAGCTGTTTGACCCTGAATGTGGCCCCTGTGAAATTTACCGTACCACATTGCCACCAAACCAGAAAGTCCACTGGGATACTCTATGCGTAGGAGGTTTGTTG TCCCTAACCCTAGCGACTGAGGCTCAGCAGAGTTTTAAGGAGTTCCTCGCCCTGTTAATGTATTATTACGTAAGTTTCAAGAAAGGTTCGAGGATTACGTTCGTCCCGC TGCCGGCGCGCAAGCCTTCCACCATCGCGTCTTTTATAGGAACTAGTCCCGTCAGCGCGGGTTTACAACGAAGCGCACCGAACATCCGCCAGGGCGTTAGTCGGCATGA CTGGGATTGCTCCAGTCCCCACGTAATAAGAAGTCCGGGACGTGAACGTAGTCATCAGACCTCCGCATACAAAGTCTTGTGATGCAGTATGCGAGTCCCGGCATCTGTT GGTGTGACGGTGGGGCATTTAACAGAATAAGCAGTATCCCCGTTCCTAGGTAGGCTATGTCGACACCACGAGTGGCCCACCGAAATTAGTTCAAGGTGGTGAAGTAGTC TGTGGATCGGTAGCCGGGTGAACGTTGAAAGACTGACCCTTGGTGTCCGCTTCCCGACTACACACAGCAAACATATCTCAATGTAAGGGAGAGCGTACCAAGCCGACCT GCATCCGGAATGAATAAGTTGAATTGATTTCTCTTCTACCGGTGCCCTAAGCGAACACTCGTCGGCGGCAGTACCACCCTGTGAGTATCCGATTTGCTTCCGTTGTGAA CGCACGATATTGCTGGGACATGACATGATTGATCCGCCCCAATTTATCTGGTAGTAGATACCCAGCCGGTGGATGGCAGCCGCTTCAGATCATATGGGTCTACGTTGAG GTGCTGGTAGCATGGTCGACACAATGTATGTGTCTGGGCGATGAGCCAGACGGCCGCTCTTCGTAGATCGTCGCGCGTGGTCATCATACAGTAAGCTCCAGTCACTCAG'
ba2h(kmer,dna)

68